# Apache Mesos HTTP API

This notebook will show how to connect to a running Master/Slave and launch commands via a simple `CommandInfo` protocol buffer.

The main goal of this notebook is to show how to interact with the new [Mesos HTTP API](https://github.com/apache/mesos/blob/master/docs/scheduler_http_api.md) in Python.

## Prerequisites

- you have RTFM (link above);
- you know how to build/run Apache Mesos locally (see the [Starting Guide](http://mesos.apache.org/gettingstarted/))
- you are familiar with Python [Requests](http://www.python-requests.org/en/latest/) framework.

## Starting Mesos

### Vagrant Up

There is a [`Vagrantfile`](https://github.com/massenz/zk-mesos/blob/develop/vagrant/Vagrantfile) provided in this repository (along with a couple of `provisioning` shell scripts) that will do the needful:

```
    cd vagrant
    vagrant up
```

### Building Mesos

If DYI is more your thing (or prefer to run from `HEAD`) then follow the instructions on the [Getting Started page](http://mesos.apache.org/gettingstarted/) and take it from there.

Nothing unusal here, start ZooKeeper (`zkServer.sh start`) then start Master/Agent nodes on `localhost`:
```
./bin/mesos-master.sh --zk=zk://localhost:2181/mesos/test --work_dir=/tmp/mesos-24 --quorum=1 --port=5051           
```

and, in another shell:
```
./bin/mesos-slave.sh --master=zk://localhost:2181/mesos/test --work_dir=/tmp/slave --port=5055
```

Then navigate to the [Mesos Web UI](http://localhost:5051) and make sure all it's working just fine.
If the above doesn't work, it's unlikely that anything in the following ever will.

Remember to adjust the Master/Agent URLs accordingly:
```
# Adjust the ports according to how you launched Mesos:
# see --port in the commands in "Prerequisites"
MASTER_URL = 'http://localhost:5051'
SLAVE_URL = 'http://localhost:5055'
```

(if the above does not work, there may be issues with `hostname` resolution - you may need to update your `/etc/hosts` accordingly - completely outside the scope of this notebook, though).


## Python Virtualenv

I always strongly recommend that folks use virtual environments when messing around with Python and installing libraries - feel free to skip this, but if you end up borking your system... **you have been warned**.

See the `requirements.txt` in this repo.

My actual `dev` virtualenv has a lot more stuff, but the above should be sufficient to get you going (and probably need strictly even less than that - YMMV); most of the dependencies above are for IPython Notebooks.

```
mkvirtualenv dev
pip install -r requirements.txt
ipython notebook
```
Then load this file in your Notebook.

Happy hacking!


# Common Imports & Useful globals

In [1]:
from __future__ import print_function

import json
import os
import pprint
import random
import requests
import sh
from threading import Thread
from time import ctime, sleep, time


# See KillTaskMessage in include/mesos/v1/scheduler/scheduler.proto
SUBSCRIBE_BODY = {
    "type": "SUBSCRIBE",
    "subscribe": {
        "framework_info": {
            "user" :  "vagrant",
            "name" :  "Example HTTP Framework"
        },
        "force" : True
    }
}

#### **NOTE**
#
# Even though framework_id is defined as "optional" in scheduler.proto, it MUST
# always be present:
#      optional FrameworkID framework_id = 1;
#
# in all Call messages, apart from the SUBSCRIBE - because we don't have an ID
# before subscribing (which is why it's defined as `optional`).


# See KillTaskMessage in include/mesos/v1/scheduler/scheduler.proto
TEARDOWN_BODY = {
    "type": "TEARDOWN",
    "framework_id": {
        "value" : None
    }
}

# See KillTaskMessage in include/mesos/v1/scheduler/scheduler.proto
KILLTASK_BODY = {
    "type": "KILL",
    "framework_id": {
        "value" : None
    },
    "kill": {
        "agent_id": {"value": None},
        "task_id": {"value": None}
    }
}


DOCKER_JSON = "../resources/container.json"
LAUNCH_JSON = "../resources/launch.json"
TASK_RESOURCES_JSON = "../resources/task_resources.json"


# Adjust the ports according to how you launched Mesos:
# see --port in the commands in "Prerequisites"
MASTER_URL = 'http://192.168.33.10:5050'
SLAVE_URL = 'http://192.168.33.11:5051'
API_V1 = '/api/v1/scheduler'
API_URL = '{}/{}'.format(MASTER_URL, API_V1)
CONTENT = 'application/json'

headers = {
    "Content-Type": CONTENT, 
    "Accept": CONTENT, 
    "Connection": "close"
}

pretty = pprint.PrettyPrinter(indent=2)

def get_json(filename):
    """ Loads the JSON from the given filename."""
    with open(filename) as jsonfile:
        lines = jsonfile.readlines()

    return json.loads("".join(lines))

These are the globals that are used to communicate with the background thread; they are currently **thread-unsafe** and may (or may not - chances of a race are pretty slim here) need to be protected with a `RLock`

In [2]:
# TODO: THIS IS THREAD-UNSAFE
terminate = False
offers = []
framework_id = None
last_heartbeat = None

## POST helper method

Sends POST request to the given URL using the `requests` library: all optional arguments passed in `**kwargs` are passed straight through to the `post()` call.

If it's not a "streaming" request (see below) and we get a `2xx` response, we return the `Response` object.

### Streaming channel

When we specify a `stream` argument in `**kwargs`, then we open a "streaming channel" to Master: this is used to subscribe a Framework.

This uses [Request's streaming API](http://www.python-requests.org/en/latest/user/advanced/#chunk-encoded-requests) for the "chunk-encoded response".

This method opens a persistent connection to the Master which will continue to receive events "callbacks" for
the lifetime of the Framework; the stream is serialized in `RecordIO` format, which essential means it
looks something like:
```
110\n
{ "type": "OFFERS", ... }224\n
{ "type": "HEARTBEAT"... }435\n
...
```
this will continue until we either tear down the connection, or send a `TEARDOWN` call (see `terminate_framework()` below).

In [3]:
def post(url, body, **kwargs):
    """ POST `body` to the given `url`.
    
        @return: the Response from the server.
        @rtype: requests.Response
    """
    import time
    print('Connecting to Master: ' + url)
    r = requests.post(url, headers=headers, data=json.dumps(body), **kwargs)
    
    if r.status_code not in [200, 202]:
        raise ValueError("Error sending request: {} - {}".format(r.status_code, r.text))
    if 'stream' in kwargs:
        # The streaming format needs some munging:
        first_line = True
        for line in r.iter_lines():
            if first_line:
                count_bytes = int(line)
                first_line = False
                continue
            body = json.loads(line[:count_bytes])
            count_bytes = int(line[count_bytes:])
            if body.get("type") == "HEARTBEAT":
                global last_heartbeat
                last_hearbeat = time.ctime()
            if body.get("type") == "ERROR":
                print("[ERROR] {}".format(body))
            # When we get OFFERS we want to see them (and eventually, use them)
            if body.get("type") == "OFFERS":
                global offers
                offers = body.get("offers")
            # We need to capture the framework_id to use in subsequent requests.
            if body.get("type") == "SUBSCRIBED":
                global framework_id
                framework_id = body.get("subscribed").get("framework_id").get("value")
                if framework_id:
                    print("Framework {} registered with Master at ({})".format(framework_id, url))
            if terminate:
                return
    return r

In [4]:
def get_framework(index=None, id=None):
    """Gets information about the given Framework.
    
       From the `/state.json` endpoint (soon to be deprecated, in favor of `/state`)
       we retrieve the Framework information.
       
       Can only specify one of either `index` or `id`.
       
       @param index: the index in the array of active frameworks
       @param id: the framework ID
       @return: the full `FrameworkInfo` structure
       @rtype: dict
    """
    if index and id:
        raise ValueError("Cannot specify both ID and Index")
    r = requests.get("{}/state.json".format(MASTER_URL))
    master_state = r.json()
    frameworks = master_state.get("frameworks")
    if frameworks and isinstance(frameworks, list):
        if index is not None and len(frameworks) > index:
            return frameworks[index]
        elif id:
            for framework in frameworks:
                if framework.get("id") == id:
                    return framework


## Warm up

The following code just checks that there is connectivity and the settings are all correct: do not move forward until this run successfully.

In [6]:
r = requests.get("{}/state.json".format(MASTER_URL))
master_state = r.json()

r = requests.get("{}/state.json".format(SLAVE_URL))
slave_state = r.json()

# If this is not true, you're in for a world of hurt:
assert master_state["version"] == slave_state["version"]
print("Mesos version running at {}".format(master_state["version"]))

# And right now there ought to be no frameworks:
assert get_framework(index=0) is None

Mesos version running at 0.24.1


# Registering a Framework

Using the HTTP API requires to run at least two separate threads: one for the "incoming" Master messages **to** the Framework (the HTTP connection we opened with the initial `SUBSCRIBE` `POST`) and another **from** the Framework to the Master to actual convey our requests (eg, accepting `OFFER`s).

We will be using the `threading` module, as this is I/O-bound and there is no CPU contention; we will run a background thread (`persistent_channel`) to receive messages from Mesos, and will use the main thread to send `requests` to Master.

The code in this Notebook **is not thread-safe**; in particular, we don't use any form of locking, as there is no real concern about races over shared data: in real production code, one should obviously protect shared data with suitable `locks` (see the [Python Multithreading documentation](https://docs.python.org/3/library/threading.html) for more details).

In [7]:
try:
    kwargs = {'stream':True, 'timeout':30}
    persistent_channel = Thread(target=post, args=(API_URL, SUBSCRIBE_BODY), kwargs=kwargs)
    persistent_channel.daemon = True
    persistent_channel.start()
    print("The background channel was started to {}".format(API_URL))
except Exception, ex:
    print("An error occurred: {}".format(ex))

The background channel was started to http://192.168.33.10:5050//api/v1/scheduler
Connecting to Master: http://192.168.33.10:5050//api/v1/scheduler


# Accepting Offers for Resources

We need a tiny amount of resources (0.1 CPU, 32 MB of RAM) to run a simple command on the Slave.

## Wait for Offers

We need to wait first for the framework to register, then to get resource offers:

In [8]:
# This code is safe to execute any number of times; it will only try to connect once.
# In other words, it's idempotent:

count = 0
while not framework_id and count < 10:
    sleep(3)
    print('.', end="")
    count += 1
    
if not framework_id:
    print("Failed to register, terminating Framework")
    close_channel()
else:
    print("Registered a Framework with ID: {}".format(framework_id))

    print("Waiting for offers...")

    count = 0
    while not offers and count < 10:
        print('.', end="")
        sleep(3)
        count += 1
        
    if not offers:
        print("Failed to obtain resources, terminating Framework")
        terminate_framework(framework_id)
        close_channel()
    else:
        print("Got offers:")
        pretty.pprint(offers)

Framework 20151009-101103-169978048-5050-6355-0000 registered with Master at (http://192.168.33.10:5050//api/v1/scheduler)
Registered a Framework with ID: 20151009-101103-169978048-5050-6355-0000
Waiting for offers...
Got offers:
{ u'offers': [ { u'agent_id': { u'value': u'20151009-101103-169978048-5050-6355-S0'},
                 u'framework_id': { u'value': u'20151009-101103-169978048-5050-6355-0000'},
                 u'hostname': u'mesos-agent',
                 u'id': { u'value': u'20151009-101103-169978048-5050-6355-O0'},
                 u'resources': [ { u'name': u'cpus',
                                   u'role': u'*',
                                   u'scalar': { u'value': 1},
                                   u'type': u'SCALAR'},
                                 { u'name': u'mem',
                                   u'role': u'*',
                                   u'scalar': { u'value': 244},
                                   u'type': u'SCALAR'},
                       

## For each slave, start simple server.

### Get the number of slaves

In [ ]:
r = requests.get("{}/state.json".format(MASTER_URL))
master_state = r.json()
number_of_slaves = master_state.get("activated_slaves")
print("Number of slaves: " + number_of_slaves)

### Start a job on each slave

In [26]:
for i in range(0, len(offers)):
    print("Starting offer ", i)
    offer = offers.get('offers')[i]
    launch_json = get_json(LAUNCH_JSON)

    task_id = str(random.randint(100, 1000))

    launch_json["accept"]["offer_ids"].append(offer["id"])
    launch_json["framework_id"]["value"] = framework_id

    task_infos = launch_json["accept"]["operations"][0]["launch"]["task_infos"][0]

    task_infos["task_id"]["value"] = task_id
    task_infos["command"]["value"] = "cd /var/local/www && /usr/bin/python -m SimpleHTTPServer 9000"
    task_infos["agent_id"]["value"] = offers.get('offers')[0]["agent_id"]["value"]
    task_infos["resources"] = get_json(TASK_RESOURCES_JSON)


    try:
        r = post(API_URL, launch_json)
        print("Result: {}".format(r.status_code))
        if r.text:
            print(r.text)
        if 200 <= r.status_code < 300:
            print("Successfully launched task {} on Agent [{}]".format(task_id, offers.get('offers')[0]["agent_id"]["value"]))
    except ValueError, err:
        print("Request failed: {}".format(err))